# Dimensionality Reduction

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as skl
import numpy as np

import seaborn as sns
sns.set(font_scale=2)

%matplotlib inline

In [ ]:
import sklearn.decomposition

In [ ]:
train = pd.read_csv('../data/training.csv')

train.head()

In [ ]:
data_columns = [column for column in train.columns if column.startswith('m')]
wavenumbers = [float(column.lstrip('m')) for column in data_columns]

output_columns = ["Ca","P","pH","SOC","Sand"]

### Why Dimensionality Reduction?

In [ ]:
train.shape

As the cell above indicates, we have more numbers observed for each datapoint than we have datapoints. This will lead many models, like linear regression, to "overfit" the data -- they will perfectly predict the outputs for data they've seen, but perform poorly on unseen data.

We can resolve this by summarizing the numbers we did observe for each datapoint with a smaller set of numbers. Since the size of the collection of numbers we're using is called the "dimension", this process is called "dimensionality reduction". For now, this notebook only covers a dimensionality reduction technique called "Principal Components Analysis", or PCA. 

### PCA - Optimal Dimensionality Reduction for Gaussians

Principal components analysis performs the best possible (i.e. information-loss-minimizing) transformation for multivariate Gaussian data (a linear mixture of independent Gaussians).

It does this by computing the covariance matrix of the data and then "diagonalizing" it -- transforming the data so that, if we calculate the covariance matrix after transforming, the values outside the diagonal are 0. This is achieved by rotating the data vectors.

This is also called decorrelation for the following reason: these values correspond to the covariances of datapoints with each other, and the correlation is just the covariance normalized between 0 and 1. Zero covariance therefore means no correlation.

Diagonalizing a matrix requires finding its eigenvectors. These eigenvectors are called the "principal components" of the data. They have the property that, if we sort them by eigenvalue, projecting our data onto the first $k$ eigenvectors gives us the $k$-dimensional projection of our data that has the highest variance. For Gaussian data, this is equivalent to preserving the information present in the input data.

In [ ]:
full_PCA = sklearn.decomposition.PCA()

In [ ]:
X = train[data_columns].as_matrix()
y = train[output_columns].as_matrix()

In [ ]:
zero_centered_X = X-np.mean(X,axis=1)[:,None]

In [ ]:
full_PCA.fit(zero_centered_X)

To see how well our PCA is performing, we calculate the remaining variance once the data has been projected onto the given component (performed by taking the dot product of the data point and the component).

This is called the "variance explained" by the component (though it might make more sense to call it the "variance retained").

The cell below plots the variance explained (as a fraction of the total variance) for all the components and then the first 120, 60, 30, and 12 components. I've done this because the fall-off is steep -- only the first few components have a variance explained that's far away from zero. This is great news, since it means we can perform really intense dimensionality reduction without fear of losing lots of useful information!

In [ ]:
total_components = len(full_PCA.explained_variance_ratio_)
plt.figure(figsize=(12,4))

for idx,num_components in enumerate([total_components,120,60,30,12]):
    plt.subplot(2,3,idx+1)
    plt.plot(np.arange(1,num_components+1,1),
             full_PCA.explained_variance_ratio_[:num_components]);
    plt.title(str(num_components)+" components",
            #fontsize='x-large',
            );
    
plt.suptitle('Fraction Variance Explained By Each Component',
             fontweight='bold',
             fontsize='xx-large',
             y=1.05)
plt.tight_layout()

The cell below plots much of the same information, but now plots the variance retained when we project the data onto the number of components on the x-axis.

In [ ]:
total_components = len(full_PCA.explained_variance_ratio_)
plt.figure(figsize=(12,4))

for idx,num_components in enumerate([total_components,120,60,30,12]):
    plt.subplot(2,3,idx+1)
    plt.plot(np.arange(1,num_components+1,1),
             np.cumsum(full_PCA.explained_variance_ratio_[:num_components])
                      );
    plt.title(str(num_components)+" components",
            #fontsize='x-large',
            );
    plt.ylim(0,1)
    
plt.suptitle('Cumulative Fraction of Variance Explained',
             fontweight='bold',fontsize='xx-large',y=1.05)
plt.tight_layout()

### Viewing the PCs

The cell below grabs the PCs selected with `indices_to_plot` and plots them in a grid. It's possible that these spectra are meaningful, and it'd be great if we could determine that!

The "transformed" data values are just the dot products between the data value and each principal component. These are called the "loadings" onto each principal component.

In [ ]:
indices_to_plot = list(range(9))
number_to_plot = len(indices_to_plot)
components_to_plot = full_PCA.components_[indices_to_plot,:]

nrows = int(np.ceil(np.sqrt(number_to_plot)))
ncols = nrows

fig,axes = plt.subplots(nrows=nrows,ncols=ncols,
             sharex=True,sharey=True,
             figsize=(20,12),
            )
for ax,component in zip(np.ravel(axes),components_to_plot):
    ax.plot(wavenumbers,component)
    
plt.suptitle('First ' +str(number_to_plot) + ' Principal Components',
            fontweight='bold',fontsize=48,y=1.1,
            );
plt.tight_layout()

explained_variance_percent = np.sum(full_PCA.explained_variance_ratio_[indices_to_plot])*100
print("these components collectively explain {:0.2f}% of the variance".format(explained_variance_percent))

### How do our reconstructions compare to the originals?

We can "reconstruct" a datapoint that we reduced to a lower dimension by multiplying each of the principal components by the loading of that datapoint onto that component. Because of the rapid rise in the cumulative variance retained, we can get excellent reconstructions while only keeping a small number of PCs.

In [ ]:
to_keep = 3

compressive_PCA = sklearn.decomposition.PCA(n_components=to_keep).fit(zero_centered_X)
explained_variance_percent = np.sum(compressive_PCA.explained_variance_ratio_)*100
print("these components collectively explain {:0.2f}% of the variance".format(explained_variance_percent))

In [ ]:
transformed_data = compressive_PCA.transform(zero_centered_X)
reconstructions = compressive_PCA.inverse_transform(transformed_data)

#### Run Repeatedly to Compare Random Examples

ProTip: enter "Command Mode" by pressing `Esc` and then hit `h` (for `h`elp) to see a list of keyboard shortcuts. One of them should let you run a cell while keeping it selected, which is useful for cells like the one below.

In [ ]:
numDataPoints = transformed_data.shape[0]

random_idx = np.random.choice(numDataPoints)
reconstruction = reconstructions[random_idx,:]
original = zero_centered_X[random_idx,:]

plt.plot(wavenumbers,reconstruction,label='recon');
plt.plot(wavenumbers,original,label='orig');
plt.suptitle('Example Reconstruction with '+str(to_keep)+' components',
            fontweight='bold',fontsize='xx-large',
            );



### Running Linear Regression on top of PCA

The cells below perform linear regression on the PCA-transformed data values.

The results are surprisingly good for this dataset! Despite throwing out 99% of the data, we still get a decent score, because PCA helped us determine the right 1% to keep. A good extension for this section would be to add in cross-validation so that the results can be fairly compared with the un-reduced data.

In [ ]:
to_keep = 12

compressive_PCA = sklearn.decomposition.PCA(n_components=to_keep).fit(zero_centered_X)
explained_variance_percent = np.sum(compressive_PCA.explained_variance_ratio_)*100
print("these components collectively explain {:0.2f}% of the variance".format(explained_variance_percent))

In [ ]:
transformed_data = compressive_PCA.transform(zero_centered_X)

In [ ]:
LR = sklearn.linear_model.LinearRegression()

In [ ]:
LR.fit(transformed_data,y)

In [ ]:
LR.score(transformed_data,y)